In [6]:
# Downgrade numpy to avoid version conflicts
!pip install numpy==1.25.0 --quiet

# Install required packages
!pip install facenet-pytorch==2.5.2 gradio torch torchvision opencv-python pillow --quiet

  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
import os
os._exit(00)

In [1]:
import numpy as np
from PIL import Image
import torch
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1
import gradio as gr

# Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

# Initialize models
mtcnn = MTCNN(keep_all=True, device=device, margin=20, post_process=True)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Face matching function
def match_faces(solo_img, group_img, threshold=0.65):
    try:
        # Convert to RGB
        solo_img = solo_img.convert("RGB")
        group_img = group_img.convert("RGB")
        image_np = np.array(group_img)

        # --- Solo face extraction (only FIRST face is used) ---
        solo_faces = mtcnn(solo_img)
        if solo_faces is None:
            return "No faces detected in solo image.", group_img

        # If multiple faces are detected in solo, pick the first one only
        if solo_faces.ndim == 4:
            solo_face = solo_faces[0].unsqueeze(0).to(device)
        elif solo_faces.ndim == 3:
            solo_face = solo_faces.unsqueeze(0).to(device)
        else:
            return "Invalid solo face detection.", group_img

        solo_embedding = resnet(solo_face)

        # --- Group faces detection ---
        group_faces = mtcnn(group_img)
        boxes, _ = mtcnn.detect(group_img)

        if boxes is None or group_faces is None:
            return "No faces detected in group image.", group_img

        total_faces = len(boxes)
        match_found = False

        # Compare each group face strictly against the solo face
        for i, box in enumerate(boxes):
            group_face_tensor = group_faces[i].unsqueeze(0).to(device)
            group_emb = resnet(group_face_tensor)

            sim_score = torch.nn.functional.cosine_similarity(group_emb, solo_embedding).item()

            # Only mark as a match if it passes strict threshold
            if sim_score > threshold:
                match_found = True
                color = (0, 255, 0)  # Green box
                label = f"Match: {sim_score*100:.1f}%"
            else:
                # Ignore all other detections (do not label or mark them)
                continue

            # Draw only matched face
            x1, y1, x2, y2 = [int(v) for v in box]
            cv2.rectangle(image_np, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image_np, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        result_text = f"Total faces in group: {total_faces}\nMatch found: {'Yes' if match_found else 'No'}"
        return result_text, Image.fromarray(image_np)

    except Exception as e:
        return f"Unexpected error: {str(e)}", group_img


# Gradio interface
iface = gr.Interface(
    fn=match_faces,
    inputs=[
        gr.Image(type="pil", label="Solo Image (only ONE person is used)"),
        gr.Image(type="pil", label="Group Image"),
        gr.Slider(minimum=0.5, maximum=0.8, step=0.01, value=0.65, label="Match Threshold (Strict)"),
    ],
    outputs=[
        gr.Textbox(label="Result"),
        gr.Image(label="Detected Face (only solo person)"),
    ],
    title="Strict Face Match Detection",
    description="Detects ONLY the person in the solo image inside the group image. Ignores all other faces, even if similar."
)

# Launch with public link
iface.launch(share=True)

Using device: cpu
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://137a9cce6a830d2dfe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
